# Project 1: Classify Images of Road Traffic Signs

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from google.colab import drive
drive .mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp /content/drive/'My Drive'/Assignment 2/trafficsigns_dataset

cp: cannot stat '/content/drive/My Drive/Assignment': No such file or directory


### Objective
- objective 1

### Dataset

Input variables:

Output variable (desired target):

### EDA